In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

X , y , coef = make_regression(n_samples=200 , n_features=2 , bias = 3 , noise=5 , coef = True)
print(X.shape , y.shape , coef)


from mpl_toolkits import mplot3d

%matplotlib qt
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection = '3d')
ax.scatter(X[:,0] , X[:,1] , y)
ax.set_xlabel("X1")
ax.set_ylabel("X2")
ax.set_zlabel("Label y")
plt.show()

(200, 2) (200,) [31.37645515 34.29748324]


In [11]:
class LinearRegressor :
    
    def predict (self , X) :
        if self.w.shape[0] != X.shape[1] :
            X = X.copy()
            ones_column = np.ones((len(X) , 1))
            X = np.concatenate([ones_column , X] , axis = 1)
        return X.dot(self.w) 
    
    def sumOfResidual (self , y , y_hat):
        return (y - y_hat).sum()
    
    def __get_gradient(self , X , y, y_hat) :
        grad = []
        for j in range(X.shape[1]) :
            g = -2 * ((y - y_hat).reshape(-1,) * X[:,j]).sum() 
            grad.append(g)
        return np.array(grad).reshape(-1,1)
            
    
    def __gradient_descent(self , X , y , epochs , learning_rate , batch_size) :
        idx = np.arange(0 , len(X))
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        
        itr = len(X) // batch_size
        
        lossList = []
        for i in range(epochs) :
            for j in range(itr) :
                X_batch , y_batch = X[j * batch_size :(j+1) * batch_size] , y[j * batch_size : (j + 1) * batch_size]
                y_hat_batch = self.predict(X_batch)
                grad = self.__get_gradient(X_batch , y_batch, y_hat_batch)
                self.w = self.w - learning_rate * grad
             
            yhat = self.predict(X)
            loss = self.loss(y , yhat)
            lossList.append(loss)
            r2 = self.r_squared(y , yhat)
            sor = self.sumOfResidual(y , yhat)
            print(f"{i + 1}/{epochs}  Loss : {loss} r2 : {r2}  sor : {sor} ")
            
        return lossList
    
    def loss(self ,y , y_hat) :
        return (y - y_hat).T.dot(y - y_hat)[0][0]
    
    def r_squared(self , y , y_hat) :
        e_method = self.loss(y , y_hat)
        mean = y.mean()
        e_baseline = self.loss(y , mean)
        return 1 - (e_method/e_baseline)
    
    def fit(self , X , y, epochs = 1000 , learning_rate = 0.01 , method = 'batch', **kwargs) :
        print(type(epochs))
        X = X.copy()
        ones_column = np.ones((len(X) , 1))
        X = np.concatenate([ones_column , X], axis = 1)
        print(X.shape)
        
        self.w = np.random.rand(X.shape[1] , 1) 
        
        if method == 'batch' :
            batch_size = X.shape[0]
        elif method == 'mini-batch' :
            if kwargs.get('batch_size') == None :
                batch_size = int(X.shape[0] * 0.25)
            else :
                batch_size = kwargs['batch_size']
        elif method == 'stochastic' :
            batch_size = 1
            
        return self.__gradient_descent(X , y ,epochs, learning_rate , batch_size)    

In [13]:
lr = LinearRegressor()
losses = lr.fit(X , y.reshape(-1,1) , epochs = 500 , learning_rate = 0.0002 , method = 'batch')
plt.plot(losses)
plt.show()

<class 'int'>
(200, 3)
1/500  Loss : 389405.2607767724 r2 : 0.16936344998989017  sor : 1101.881511717465 
2/500  Loss : 325586.44515661907 r2 : 0.30549474088902295  sor : 953.1599292963772 
3/500  Loss : 272407.63053522346 r2 : 0.41892994980897924  sor : 821.5682350985928 
4/500  Loss : 228087.55203761265 r2 : 0.5134686754183807  sor : 705.2814612190341 
5/500  Loss : 191144.2473702212 r2 : 0.5922720769792177  sor : 602.660333988368 
6/500  Loss : 160344.60212852719 r2 : 0.6579704987572472  sor : 512.2329362245741 
7/500  Loss : 134662.42473793926 r2 : 0.7127528999551975  sor : 432.678143163234 
8/500  Loss : 113243.6054524013 r2 : 0.7584411737117972  sor : 362.81066307467063 
9/500  Loss : 95377.15935826206 r2 : 0.7965519149867714  sor : 301.5675294968813 
10/500  Loss : 80471.1570748893 r2 : 0.8283477625477556  sor : 247.99590644699722 
11/500  Loss : 68032.71592913785 r2 : 0.8548800796001785  sor : 201.24208105872466 
12/500  Loss : 57651.36471357873 r2 : 0.8770244382586471  sor : 1

In [14]:
ypred = lr.predict(X)
print(ypred.shape)

(200, 1)


In [18]:
plt.figure(figsize = (6 , 6))
ax = plt.axes(projection = '3d')
ax.scatter(X[:,0] , X[:,1] , y)
ax.set_xlabel("X1")
ax.set_ylabel("X2")
ax.set_zlabel("Label y")

ax.plot_surface(X[:,0] , X[:,1] , ypred)


In [24]:
f1 = np.linspace(X[:,0].min() , X[:,0].max() , 50)
f2 = np.linspace(X[:,1].min() , X[:,1].max() , 50)

f1 , f2 = np.meshgrid(f1 , f2)


f1 = f1.reshape(-1 , 1) 
f2 = f2.reshape(-1 , 1)

x = np.concatenate([f1 , f2] , axis = 1)
print(x.shape)

(2500, 2)


In [25]:
ypred = lr.predict(x)
print(ypred.shape)

(2500, 1)


In [36]:
%matplotlib qt
plt.figure(figsize = (6 , 6))
ax = plt.axes(projection = '3d')
ax.scatter(X[:,0] , X[:,1] , y )
ax.set_xlabel("X1")
ax.set_ylabel("X2")
ax.set_zlabel("Label y")

ax.scatter(f1 , f2 , ypred , s = 5 , c = ypred , cmap = plt.cm.coolwarm , alpha = 0.5)
plt.show()
